**Load All Datasets from GitHub**


In [ ]:
# Step 0: Install rasterio
!pip install rasterio

# Step 1: Import libraries
import pandas as pd
import rasterio
import matplotlib.pyplot as plt

# Step 2: Define GitHub raw base URL
base_url = "https://raw.githubusercontent.com/Dr-Isam-ALJAWARNEH/fds-project-geoairmob/main/Datasets/"

# Step 3: Load CSV datasets
mobility_df = pd.read_csv(base_url + "AE_Region_Mobility_Report_2020_2022_combined.csv")
openaq_df = pd.read_csv(base_url + "combined_openaq_measurements.csv")
ericsson_df = pd.read_csv(base_url + "Merged_Ericsson_Mobility_Data.csv")
landsat_df = pd.read_csv(base_url + "Landsat_SRTM_Merged_Metadata.csv")

print("✅ Datasets loaded:")
print("Mobility:", mobility_df.shape)
print("OpenAQ:", openaq_df.shape)
print("Ericsson:", ericsson_df.shape)
print("Landsat:", landsat_df.shape)

# Step 4: Visualize raster
tif_url = "/vsicurl/https://github.com/Dr-Isam-ALJAWARNEH/fds-project-geoairmob/raw/main/Datasets/UAE_NO2_SO2_Aerosol_Combined_2020_2022.tif"

with rasterio.open(tif_url) as src:
    img = src.read(1)
    plt.imshow(img, cmap='plasma')
    plt.title("NO₂/SO₂/Aerosol - UAE (2020–2022)")
    plt.colorbar()
    plt.show()



# **Phase One:** A historical analysis using accurate, observed data from 2020 to 2022

**Install Geohash Library in Colab**

In [ ]:
!pip install geohash2


**Generate Geohashes for OpenAQ Data**

In [ ]:
openaq_df.columns


In [ ]:
import geohash2

# Recalculate geohash with 5-character precision (~3km)
openaq_df['geohash'] = openaq_df.apply(
    lambda row: geohash2.encode(row['latitude'], row['longitude'], precision=5),
    axis=1
)


In [ ]:
# Group by geohash and pollutant type (e.g., NO2, PM2.5)
pollution_summary = openaq_df.groupby(['geohash', 'parameter'])['value'].mean().unstack()

# Clean table
pollution_summary = pollution_summary.reset_index()
pollution_summary.head()


In [ ]:
# Count unique geohashes
unique_pollution_geohashes = pollution_summary['geohash'].nunique()

print(f"✅ Number of unique geohash zones in the pollution dataset: {unique_pollution_geohashes}")


In [ ]:
print("🔍 Sample geohash zones:", pollution_summary['geohash'].unique()[:18])


**View Mobility Data**

In [ ]:
mobility_df.columns
mobility_df.head()


In [ ]:
import numpy as np

# Format: (lat_min, lat_max, lon_min, lon_max)
city_bboxes = {
    "Dubai": (25.0, 25.3, 55.2, 55.4),
    "Abu Dhabi": (24.2, 24.6, 54.3, 54.6),
    "Sharjah": (25.25, 25.45, 55.35, 55.5),
    "Al Ain": (24.1, 24.3, 55.6, 55.8),
    "Ajman": (25.35, 25.45, 55.45, 55.5),
    "Fujairah": (25.05, 25.2, 56.25, 56.4),
    "Ras Al Khaimah": (25.7, 25.85, 55.85, 56.0),
    "Umm Al Quwain": (25.5, 25.6, 55.5, 55.6),
}


In [ ]:
# Define place_id to city mapping
placeid_to_city = {
    "ChIJvRKrsd9IXj4RpwoIwFYv0zM": "Dubai",
    "ChIJGczaTT5mXj4RBNmakTvGr4s": "Abu Dhabi",
    "ChIJHwyp6rZXXz4RerixWbtcrRE": "Sharjah",
    "ChIJRcbZaklDXz4R6SkAK7_QznQ": "Al Ain",
    "ChIJX7kokD0y9D4RvDyz2xuxwaY": "Ajman",
    "ChIJpwnSTA5x9j4RD-KEpgxnnrk": "Fujairah",
    "ChIJ0Zp9hY5ZXz4RZlq4HHZvx2g": "Ras Al Khaimah",
    "ChIJP-14gXD99T4RGDyAmyB_vKE": "Umm Al Quwain"
}

# Assign city based on place_id
mobility_df['city'] = mobility_df['place_id'].map(placeid_to_city)


In [ ]:
def get_random_coords(city):
    if city in city_bboxes:
        lat_min, lat_max, lon_min, lon_max = city_bboxes[city]
        lat = np.random.uniform(lat_min, lat_max)
        lon = np.random.uniform(lon_min, lon_max)
        return pd.Series([lat, lon])
    return pd.Series([None, None])

# Apply to each row
mobility_df[['latitude', 'longitude']] = mobility_df['city'].apply(get_random_coords)


In [ ]:
import geohash2

# Compute geohash from synthetic lat/lon
mobility_df['geohash'] = mobility_df.apply(
    lambda row: geohash2.encode(row['latitude'], row['longitude'], precision=5),
    axis=1
)


In [ ]:
# Create composite mobility score
mobility_df['mobility_score'] = mobility_df[
    ['transit_stations_percent_change_from_baseline', 'workplaces_percent_change_from_baseline']
].mean(axis=1)

# Group by geohash
mobility_summary = mobility_df.groupby('geohash')['mobility_score'].mean().reset_index()


In [ ]:
# Count unique geohash zones in mobility dataset
unique_mobility_geohashes = mobility_summary['geohash'].nunique()

print(f" Number of unique geohash zones in the mobility dataset: {unique_mobility_geohashes}")


In [ ]:
print(" Sample mobility geohash zones:", mobility_summary['geohash'].unique()[:217])


In [ ]:
# Convert to lowercase to ensure consistency
pollution_summary['geohash'] = pollution_summary['geohash'].str.lower()
mobility_summary['geohash'] = mobility_summary['geohash'].str.lower()

# Get unique geohash sets
pollution_geos = set(pollution_summary['geohash'].unique())
mobility_geos = set(mobility_summary['geohash'].unique())

# Get the intersection
common_geos = pollution_geos & mobility_geos

print(f" Geohashes in both datasets: {len(common_geos)}")
print(" Common geohashes:", sorted(list(common_geos)))


In [ ]:
filtered_df = merged_df[merged_df['geohash'].isin(common_geos)].copy()
print(filtered_df.shape)
filtered_df.head(10)


In [ ]:
merged_df.dtypes


In [ ]:
merged_df.to_csv("uae_mobility_pollution_merged.csv", index=False)


**Convert to Table Abstraction**

In [ ]:
from datascience import *

# Use your actual merged DataFrame
joined_table = Table().from_df(merged_common)

# Preview the table
joined_table.show(5)



In [ ]:
# Show all available columns
joined_table.labels


## **EDA**

**Summary Stats Table**




In [ ]:
# Select columns for summary
summary_table = joined_table.select('no2', 'pm25', 'pm10', 'mobility_score')

# Show descriptive stats
summary_table.stats()


Zones Exceeding NO₂ Limit (WHO = 40 µg/m³)

In [ ]:
# Find high-pollution zones
high_no2 = joined_table.where('no2', are.above(40))

# How many?
print(" Zones exceeding NO₂ threshold:", high_no2.num_rows)

# Preview
high_no2.select('geohash', 'no2', 'mobility_score').show(5)


thqf4 is a critical hotspot — both highest NO₂ and high mobility change.

**Visualize Pollution vs Mobility (Scatterplot)**

In [ ]:
import matplotlib.pyplot as plt

# Filter out missing values manually
filtered = joined_table.where('no2', are.not_equal_to(float('nan')))
filtered = filtered.where('mobility_score', are.not_equal_to(float('nan')))

# Extract columns as arrays
x = filtered.column('mobility_score')
y = filtered.column('no2')

# Plot
plt.figure(figsize=(10, 7))
plt.scatter(x, y, color='teal', edgecolor='black', s=100, alpha=0.8)

# Add WHO guideline line
plt.axhline(40, color='red', linestyle='--', linewidth=2, label='WHO NO₂ Limit (40 µg/m³)')

# Labels and title
plt.title("NO₂ vs Mobility Score (UAE Urban Zones)", fontsize=16)
plt.xlabel("Mobility Score (% Change from Baseline)", fontsize=12)
plt.ylabel("NO₂ Concentration (µg/m³)", fontsize=12)

# Legend and formatting
plt.legend(fontsize=10)
plt.grid(True)
plt.tight_layout()
plt.show()


Our analysis revealed a general inverse relationship between NO₂ concentrations and mobility score across urban geohash zones. However, some zones (e.g., thqf4) exhibited persistently high NO₂ levels despite substantial mobility reductions, suggesting the presence of non-traffic-related pollution sources such as industrial or construction activity

**Correlation Heatmap**

In [ ]:
# Select relevant columns
corr_table = joined_table.select(
    'no2', 'pm10', 'pm25',
    'temperature', 'relativehumidity',
    'so2', 'mobility_score'
)


In [ ]:
corr_df.isna().sum()


In [ ]:
# Step 1: Keep only columns with at least 5 non-NaN values
valid_corr_df = corr_df.loc[:, corr_df.notna().sum() >= 5]

# Step 2: Drop rows that still contain NaNs
valid_corr_df = valid_corr_df.dropna()

# Step 3: Plot the heatmap
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
sns.heatmap(valid_corr_df.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title("📊 Correlation Heatmap (Filtered)", fontsize=14)
plt.tight_layout()
plt.show()


The heatmap reveals a moderate inverse correlation (r = -0.56) between mobility score and NO₂ concentrations, supporting the role of traffic-related emissions in shaping urban air quality. However, weaker or absent correlations with PM10 and SO₂ suggest the presence of non-mobility-related pollution sources, underscoring the need for multi-sectoral air quality interventions.